# Knet CNN Example

In [1]:
# using Pkg; pkg"add Knet MLDatasets"
using Knet, MLDatasets
True=true # so we can read the python params
include("common/params.py")

true

In [2]:
println("OS: ", Sys.KERNEL)
println("Julia: ", VERSION)
println("Knet: ", Pkg.dependencies()[Base.UUID("1902f260-5fb4-5aff-8c31-6271790ab950")].version)
println("GPU: ", replace(read(`nvidia-smi --query-gpu=name --format=csv,noheader`,String),'\n'=>", "))

OS: Linux
Julia: 1.5.0
Knet: 1.4.0
GPU: GeForce GTX 1060 with Max-Q Design, 


In [3]:
# define model
function initmodel(; atype=KnetArray, dtype=Float32, winit=xavier, binit=zeros)
    w(dims...)=atype(winit(dtype,dims...))
    b(dims...)=atype(binit(dtype,dims...))
    return Any[
        w(3,3,3,50), b(1,1,50,1),
        w(3,3,50,50), b(1,1,50,1),
        w(3,3,50,100), b(1,1,100,1),
        w(3,3,100,100), b(1,1,100,1),
        w(512,6400), b(512,1),
        w(10,512), b(10,1)
    ]
end;

In [4]:
# define loss and its gradient
function predict(w,x; pdrop=(0,0))
    convbias(x,w,b) = conv4(w,x;padding=1) .+ b
    fc(x,w,b) = w * mat(x) .+ b;
    x = relu.(convbias(x,w[1],w[2]))
    x = relu.(pool(convbias(x,w[3],w[4])))
    x = dropout(x,pdrop[1])
    x = relu.(convbias(x,w[5],w[6]))
    x = relu.(pool(convbias(x,w[7],w[8])))
    x = dropout(x,pdrop[1])
    x = relu.(fc(x,w[9],w[10]))
    x = dropout(x,pdrop[2])
    return fc(x,w[11],w[12])
end

loss(w,x,y;o...)=nll(predict(w,x;o...),y) # nll: negative log likelihood
lossgradient = grad(loss);

In [5]:
# load data
xtrn, ytrn = CIFAR10.traindata(); xtrn = convert(KnetArray{Float32},xtrn)
xtst, ytst = CIFAR10.testdata(); xtst = convert(KnetArray{Float32},xtst)
dtrn = minibatch(xtrn,ytrn,BATCHSIZE,shuffle=true,xtype=KnetArray{Float32})
dtst = minibatch(xtst,ytst,BATCHSIZE,shuffle=false,xtype=KnetArray{Float32})
for d in (xtrn,ytrn,xtst,ytst); println(summary(d)); end

32×32×3×50000 KnetArray{Float32,4}
50000-element Array{Int64,1}
32×32×3×10000 KnetArray{Float32,4}
10000-element Array{Int64,1}


In [6]:
# prepare for training
model = optim = nothing; GC.gc(true) # Clear memory from last run
model = initmodel()
optim = optimizers(model, Momentum; lr=LR, gamma=MOMENTUM);

┌ Warning: optimizers is deprecated, use sgd, adam etc. instead.
└ @ Knet.Train20 /home/deniz/.julia/dev/Knet/src/train20/update.jl:598


In [7]:
# cold start
@time for (x,y) in dtrn
    grads = lossgradient(model, x, y; pdrop=(0.25,0.5))
    update!(model, grads, optim)
end

 22.001135 seconds (39.44 M allocations: 1.999 GiB, 2.86% gc time)


In [8]:
# prepare for training
model = optim = nothing; GC.gc(true) # Clear memory from last run
model = initmodel()
optim = optimizers(model, Momentum; lr=LR, gamma=MOMENTUM);

In [9]:
# 159s
@info("Training...")
@time for epoch in 1:EPOCHS
    @time for (x,y) in dtrn
        grads = lossgradient(model, x, y; pdrop=(0.25,0.5))
        update!(model, grads, optim)
    end
end

┌ Info: Training...
└ @ Main In[9]:2


  9.981000 seconds (3.40 M allocations: 180.850 MiB, 1.54% gc time)
 10.001566 seconds (3.40 M allocations: 180.621 MiB, 1.52% gc time)
 10.040050 seconds (3.40 M allocations: 180.862 MiB, 1.55% gc time)
 10.338632 seconds (3.40 M allocations: 180.798 MiB, 1.63% gc time)
 10.792743 seconds (3.40 M allocations: 180.749 MiB, 1.73% gc time)
 10.497737 seconds (3.40 M allocations: 180.864 MiB, 1.68% gc time)
 10.485780 seconds (3.40 M allocations: 180.810 MiB, 1.67% gc time)
 10.392641 seconds (3.40 M allocations: 180.971 MiB, 1.68% gc time)
 10.274777 seconds (3.40 M allocations: 180.951 MiB, 1.69% gc time)
 10.284816 seconds (3.40 M allocations: 180.708 MiB, 1.72% gc time)
103.091919 seconds (34.05 M allocations: 1.766 GiB, 1.64% gc time)


In [10]:
# test accuracy 77.54
@time accuracy(model,dtst,predict)

┌ Warning: accuracy(weights,data,func; o...) is deprecated, please use accuracy(x->func(weights,x); data=data, o...) instead.
└ @ Knet.Ops20 /home/deniz/.julia/dev/Knet/src/ops20/loss.jl:227
┌ Warning: accuracy(model,data; o...) is deprecated, please use accuracy(model; data=data, o...)
└ @ Knet.Ops20 /home/deniz/.julia/dev/Knet/src/ops20/loss.jl:205


  1.908039 seconds (3.76 M allocations: 184.310 MiB, 2.54% gc time)


0.7825506343200535